In [1]:
from keras.layers import Import, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preproessing.image import ImageDataGenerator

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


ImportError: cannot import name 'Import' from 'keras.layers' (/anaconda3/envs/computer-vision/lib/python3.7/site-packages/keras/layers/__init__.py)